### Write well, Im going to be using this for a long time

#### Data we need:
user input:
- investment amount, trading pair -> amt0, amt1
- start time and end time
- time period that you assume fixed swap price, swap volumes or liquidity positions
- upper and lower price
- pool_fee_rate

data from api:
- cprice of each time period (tick, 1.0001 ** i)
- L_pool at each time period at specific pool_fee_rate (liquidity?, or simply total X tokens + Y tokens in USD)
- Swap volume at each time period at specific pool_fee_rate (volumeUSD?)
- Gas cost to mint at each time period

--------------------------------------------------------------------------------------------------------------

#### Fees
The liquidity amount is calculated from the following numbers that describe a position: 
- amount of token 0 (amt0), amount of token 1 (amt1), 
- price (as x token 1's per token 0) at the upper limit of the position (upper), 
- price at the lower limit of the position (lower) 
- and the current swap price (cprice). 

Then liquidity (L_you?) for a position is calculated as follows:

Case 1: cprice <= lower
- liquidity = amt0 * (sqrt(upper) * sqrt(lower)) / (sqrt(upper) - sqrt(lower))

Case 2: lower < cprice <= upper
- liquidity is the min of the following two calculations:
- amt0 * (sqrt(upper) * sqrt(cprice)) / (sqrt(upper) - sqrt(cprice))
- amt1 / (sqrt(cprice) - sqrt(lower))

Case 3: upper < cprice
- liquidity = amt1 / (sqrt(upper) - sqrt(lower))

Resources
- liquidity can use this code: https://github.com/JNP777/UNI_V3-Liquitidy-amounts-calcs/blob/main/UNI_v3_funcs.py

Fee is calculated by:
- Fee income = (L_you/L_pool) * swap volume under fixed time period * pool_fee_rate/100
- L_you also should be for that specific ticks only, not the whole amount you provided for. Its not linear, its calculated from the 3 cases above
- Does Case1 and Case3's fee be 0 regardless?


reference: https://uniswapv3.flipsidecrypto.com/
- check my numbers with the reference from the website

----------------------------------------------------------------------------------------

#### Impermanent Loss (is this v2 or v3)
- IL (in %) = (2 sqrt(p) / (p+1) ) - 1
- where p = r_t1/r_t2
- and r_t is a price in b at time 1
- Net $ loss = total asset value in dollars at stake time * IL (in%)

reference: https://chainbulletin.com/impermanent-loss-explained-with-examples-math/#:~:text=Impermanent%20loss%20is%20the%20difference,is%20equal%20to%20200%20DAI

--------------------------------------------------------------------------------------------------------------

#### Other cost

Gas_costs_mint = 500000 gwei * gas_price at that time (??? double check actual cost)

### PNL/APR
-PNL = Acumulated Fees_accrued (dolar value at generation) - IL - Gas_costs_mint

-APR = PNL/Initial_capital*(age of the position / year time)

--------------------------------------------------------------------------------------------------------------

## Dependencies

In [1]:
import requests
import json
import pandas as pd
import math
import numpy as np

## Main Functions

In [2]:
# function to use requests.post to make an API call to the subgraph url
def run_query(q):

    # endpoint where you are making the request
    request = requests.post('https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
                            '',json={'query': q})
    if request.status_code == 200:
        return request
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))
        
        
# turns requests into dataframe        
# def results_to_df(query_result):
#     json_data_ = json.loads(query_result.text)
#     df_data_ = json_data_['data']['pools']
#     df_ = pd.DataFrame(df_data_)

#     return df_

In [3]:
def get_token_id(symbol):
    
    # default should be first:10, in case there are more than 1 coins with the same symbol
    query_ = """ 
    {{
      tokens(first:1, where:{{symbol: "{}"}}) {{
        id
        symbol
        name
      }}
    }}""".format(symbol)
    
    # run query
    query_result_ = run_query(query_)
    json_data_ = json.loads(query_result_.text)
    
    print(' ')
    print('get_token_id: {}'.format(symbol))
    print(json_data_)
    
    # make sure only return 1 object
    if len(json_data_['data']['tokens']) == 1:
        token_id_ = json_data_['data']['tokens'][0]['id']
        return token_id_
        
    else:
        print(json_data_['data'])
        raise Exception('Returned number of token_ids != 1')

        
def get_pool_id(token0_id, token1_id, feeTier):
    query_ = """
    {{
      pools(first: 10, 
        where:{{token0: "{}",
        token1: "{}",
        feeTier:"{}" }}) 
      {{
        id
        token0{{symbol}}
        token1{{symbol}}
        feeTier
      }}
    }}""".format(token0_id, token1_id, feeTier)
    
    
    # run query
    query_result_ = run_query(query_)
    json_data_ = json.loads(query_result_.text)
    
    print('\n get_pool_id for feeTier: {}'.format(feeTier))
    print(json_data_)
    
    # make sure there is only 1 pool that matches exactly
    if len(json_data_['data']['pools']) == 1:
        pool_id_ = json_data_['data']['pools'][0]['id']
        return pool_id_
    else:
        print(json_data_['data'])
        raise Exception('Returned number of token_ids != 1')

        
    return json_data_

In [4]:
def get_poolDayDatas(pool_id, num_datapoints=1000):
    # input: pool_id
    # num_datapoints (must be multiple of max_request_)
    
    max_request_ = 1000
    quotient_ = math.floor(num_datapoints/max_request_)
#     remainder_ = num_datapoints%max_request_
            
    query_base_ = '''
    {{
      poolDayDatas(first:{},
      skip: {},
        where:{{ pool: "{}" }},
      orderBy:date,
      orderDirection: desc) 
      {{
        date
        tick
        liquidity
        volumeUSD
        pool{{
            token0{{
                symbol
            }}
            token1{{
                symbol
            }}
        }}
      }}
    }}'''
    
    poolDayDatas_array_ = []
    
    # query loop
    for i in range(quotient_):
        q_first_ = max_request_
        q_next_ = i*max_request_
        query_ = query_base_.format(q_first_, q_next_, pool_id)
        query_result_ = run_query(query_)
        json_data_ = json.loads(query_result_.text)
#         print(json_data_)
        poolDayDatas_array_ += json_data_['data']['poolDayDatas']
    
    print(' ')
    print('\n Queried PoolDayDatas, total of {} datapoints'.format(str(len(poolDayDatas_array_))))
#     print('example:')
#     print(poolDayDatas_array_[0])
    
    # array to dataframe
    df_ = pd.json_normalize(poolDayDatas_array_)
    df_.drop_duplicates(subset=['date']) 
    
    return df_

In [5]:
def get_poolHourDatas(pool_id, num_datapoints=3000):
    # input: pool_id
    # num_datapoints (must be multiple of max_request_)
    
    max_request_ = 1000
    quotient_ = math.floor(num_datapoints/max_request_)
#     remainder_ = num_datapoints%max_request_
            
    query_base_ = '''
    {{
      poolHourDatas(first:{},
      skip: {},
        where:{{ pool: "{}" }},
      orderBy:periodStartUnix,
      orderDirection: desc) 
      {{
        periodStartUnix
        pool{{
            token0{{
                symbol
            }}
            token1{{
                symbol
            }}
        }}
        liquidity
        sqrtPrice
        token0Price
        token1Price
        tick
        feeGrowthGlobal0X128
        feeGrowthGlobal1X128
        tvlUSD
        volumeToken0
        volumeToken1
        volumeUSD
        feesUSD
        txCount
        open
        high
        low
        close
      }}
    }}'''
    
    poolDayDatas_array_ = []
    
    # query loop
    for i in range(quotient_):
        q_first_ = max_request_
        q_next_ = i*max_request_
        query_ = query_base_.format(q_first_, q_next_, pool_id)
        query_result_ = run_query(query_)
        json_data_ = json.loads(query_result_.text)
#         print(json_data_)
        try:
            poolDayDatas_array_ += json_data_['data']['poolHourDatas']
        except Exception:
#             print('.. Pass')
            pass
    
    print(' ')
    print('\n Queried poolHourDatas, total of {} datapoints'.format(str(len(poolDayDatas_array_))))
#     print('example:')
#     print(poolDayDatas_array_[0])
    
    # array to dataframe
    df_ = pd.json_normalize(poolDayDatas_array_)
#     df_.drop_duplicates(subset=['periodStartUnix']) # TODO: BUGGGG ??
    
    return df_

In [6]:
def get_swaps(pool_id, time_start='1627369200', time_end='1623772800', num_datapoints=6000):
    # input: pool_id
    # num_datapoints (must be multiple of max_request_)
    
    max_request_ = 1000
    quotient_ = math.floor(num_datapoints/max_request_)
#     remainder_ = num_datapoints%max_request_
           
    query_base_ = '''
    {{
      swaps(first:{}, skip: {},
            where:{{ pool: "{}",
            timestamp_lt: "{}",
            timestamp_gt: "{}"}},
          orderBy:timestamp,
          orderDirection: desc){{
        transaction {{
          blockNumber
          timestamp
          gasUsed
          gasPrice
        }}
        id
        timestamp
        tick
        amount0
        amount1
        amountUSD
        sqrtPriceX96
      }}
    }}'''
    
    swap_arrays_ = []
    
    # query loop
    for i in range(quotient_):
        q_first_ = max_request_
        q_next_ = i*max_request_
        query_ = query_base_.format(q_first_, q_next_, pool_id, time_start, time_end)
        query_result_ = run_query(query_)
        json_data_ = json.loads(query_result_.text)
        
        try:
            swap_arrays_ += json_data_['data']['swaps']
            
        except Exception:
            pass
        
    print('Queried Swaps, total of {} datapoints'.format(str(len(swap_arrays_))))
    df_ = pd.json_normalize(swap_arrays_)
    
    # next time start, if at the edge, then we keep looping and skipping
    if len(swap_arrays_) != 0:
        # last element of timestamp, add 1 so next iterations still includes it
        next_time_start_ = str( int(df_['timestamp'][df_.index[-1]]) + 1 )
    else:
        next_time_start_ = time_start
            
            
    return df_, next_time_start_


# get_swap can only request 6000 datapoints at the time. this is to loop get_swap to get more data
def get_swaps_loop(pool_id, time_start='1627369200', time_end='1623772800',  num_datapoints= 150000):
    
    print('time_start: {}, time_end: {}'.format(time_start, time_end))
    
    max_num_query = 6000
    num_iterations = math.floor(num_datapoints/max_num_query) + 3 # add 3 just in case
    
    next_time_start_ = time_start
    for i in range(num_iterations):
        
        print('next_time_start_: {}'.format(next_time_start_))
        
        df_, next_time_start_ = get_swaps(pool_id, next_time_start_, time_end, num_datapoints=max_num_query)
        
        if i == 0:
            df_all_ = df_
        else:
            df_all_ = df_all_.append(df_)
        
    # drop duplicates
    df_all_ = df_all_.drop_duplicates(subset=['id'])
    
    # reset index
    df_all_ = df_all_.reset_index(drop=True)
    
    return df_all_

In [7]:
def merge_poolHourData_swaps_all(df_poolHourDatas_, df_swaps_all_):
    
    
    return df_merged
    

### Test running

In [8]:
TOKEN0 = 'DAI'
TOKEN1 = 'WETH'
feeTier = '3000' 

In [9]:
# Get token_id > Get pool_id > Get PoolDayDatas > Get swap data > Merge Swap data (VolumeUSD, txCount - for checking)

# Indicate Tokens and FeeTier
token0_id = get_token_id(TOKEN0)
token1_id = get_token_id(TOKEN1)
pool_id = get_pool_id(token0_id, token1_id, feeTier)

 
get_token_id: DAI
{'data': {'tokens': [{'id': '0x6b175474e89094c44da98b954eedeac495271d0f', 'name': 'Dai Stablecoin', 'symbol': 'DAI'}]}}
 
get_token_id: WETH
{'data': {'tokens': [{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2', 'name': 'Wrapped Ether', 'symbol': 'WETH'}]}}

 get_pool_id for feeTier: 3000
{'data': {'pools': [{'feeTier': '3000', 'id': '0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8', 'token0': {'symbol': 'DAI'}, 'token1': {'symbol': 'WETH'}}]}}


In [10]:
# Get poolHourDatas
df_poolHourDatas = get_poolHourDatas(pool_id, num_datapoints=10000)

# Get Swap Datas within the poolHourDatas timeframe
time_start = df_poolHourDatas['periodStartUnix'][0]
time_end = df_poolHourDatas['periodStartUnix'][df_poolHourDatas.index[-1]]

 

 Queried poolHourDatas, total of 2018 datapoints


In [11]:
# Get Swaps data
df_swaps_all = get_swaps_loop(pool_id, time_start, time_end,  num_datapoints= 150000)

time_start: 1627466400, time_end: 1620158400
next_time_start_: 1627466400
Queried Swaps, total of 6000 datapoints
next_time_start_: 1625319090
Queried Swaps, total of 6000 datapoints
next_time_start_: 1624018665
Queried Swaps, total of 6000 datapoints
next_time_start_: 1622868122
Queried Swaps, total of 6000 datapoints
next_time_start_: 1622123778
Queried Swaps, total of 6000 datapoints
next_time_start_: 1621726742
Queried Swaps, total of 6000 datapoints
next_time_start_: 1621277495
Queried Swaps, total of 5943 datapoints
next_time_start_: 1620159676
Queried Swaps, total of 1 datapoints
next_time_start_: 1620159676
Queried Swaps, total of 1 datapoints
next_time_start_: 1620159676
Queried Swaps, total of 1 datapoints
next_time_start_: 1620159676
Queried Swaps, total of 1 datapoints
next_time_start_: 1620159676
Queried Swaps, total of 1 datapoints
next_time_start_: 1620159676
Queried Swaps, total of 1 datapoints
next_time_start_: 1620159676
Queried Swaps, total of 1 datapoints
next_time_

In [12]:
# Saving 
# Suffix
SETTINGS = '{}-{}-{}-timestamp-{}-{}.csv'.format(TOKEN0, TOKEN1, feeTier, time_start, time_end)

In [13]:
df_swaps_all.to_csv('../data/df_swaps_all_'+SETTINGS)
df_poolHourDatas.to_csv('../data/df_poolHourDatas_'+SETTINGS)

### Merge Data

In [14]:
# Match timestamp with hour period, and assign to df_swaps_all['periodStartUnix']
def compute_periodStartUnix(row_):
    return row_['timestamp'] - (row_['timestamp'] % 3600)
def compute_periodEndUnix(row_):
    return row_['periodStartUnix'] + 3600

df_swaps_all['timestamp'] = df_swaps_all['timestamp'].astype(int)    
df_swaps_all['periodStartUnix'] = df_swaps_all.apply(lambda row: compute_periodStartUnix(row), axis=1)                       
df_swaps_all['periodEndUnix'] = df_swaps_all.apply(lambda row: compute_periodEndUnix(row), axis=1)                       

df_swaps_all['periodStartUnix'] = df_swaps_all['periodStartUnix'].astype(int)        
df_swaps_all['periodEndUnix'] = df_swaps_all['periodEndUnix'].astype(int)        

In [15]:
# Create swaps_txCount to compare with txCount in poolHourDatas to check integrity
df_swaps_all['swaps_txCount'] = 1

In [16]:
# Groupby->Sum based on periodStartUnix, specify columns to sum at GROUPBY_COLS
GROUPBY_COLS = ['periodStartUnix','amount0', 'amount1', 'amountUSD', 'swaps_txCount']
df_swaps_to_merge = df_swaps_all[GROUPBY_COLS]
df_swaps_to_merge = df_swaps_to_merge.astype({'periodStartUnix': 'int',
                                             'amount0':'float','amount1':'float', 
                                              'amountUSD':'float', 'swaps_txCount':'int'})
df_swaps_to_merge = df_swaps_to_merge.groupby(by=['periodStartUnix']).sum()

In [17]:
# Merge df_swaps_all (groupby) with df_poolHourDatas
df_poolHourDatas['periodStartUnix'] = df_poolHourDatas['periodStartUnix'].astype(int)
df_merged = df_poolHourDatas.merge(df_swaps_to_merge, how='left', on='periodStartUnix')
df_merged['txCount'] = df_merged['txCount'].astype(int)

In [18]:
# Saving data
df_merged.to_csv('../data/df_merged_tmp_'+SETTINGS)

In [19]:
watch_list = ['periodStartUnix',  
              'txCount', 'swaps_txCount', # check data integrity
              'amount0', 'amount1', 'amountUSD', # swaps data
              'tick', 'liquidity', 'sqrtPrice', 'tvlUSD', # pool data at that hour
              'pool.token0.symbol', 'pool.token1.symbol', # token data
              'token0Price', 'token1Price'
             ]
df_merged[watch_list]

,periodStartUnix,txCount,swaps_txCount,amount0,amount1,amountUSD,tick,liquidity,sqrtPrice,tvlUSD,pool.token0.symbol,pool.token1.symbol,token0Price,token1Price
0,1627466400,6,NaN,NaN,NaN,NaN,-77418,2257757512531221050660252,1651413884466706248548127541,28342312.47685185046280753439628505,DAI,WETH,2301.692508394563049389060612506888,0.0004344628990852921495847765155841678
1,1627462800,13,11.0,299616.218709,-1.304435e+02,299557.018495,-77395,2258385571433368075249753,1653332807396390538189673451,28284094.88822451823263508880155719,DAI,WETH,2296.352739688466583961075436184692,0.0004354731669559026211691942710914797
2,1627459200,23,16.0,-105151.549893,4.687934e+01,671187.484515,-77340,2257382392731291728666040,1657909038401687840269395027,28086638.89715616303546634557186448,DAI,WETH,2283.693254559756255552465593941893,0.0004378871803397155858037200186778924
3,1627455600,31,27.0,753317.520533,-3.305345e+02,753167.996064,-77360,2256221622137200723238932,1656280685119642923297705596,28094257.6626257711941130690799759,DAI,WETH,2288.185835338192696952740469462558,0.0004370274409343157703395490434323795
4,1627452000,17,17.0,-879730.423181,3.878647e+02,881181.264433,-77220,2261588786047650735673230,1667896272296713929624373234,27821144.05932027138316057295951014,DAI,WETH,2256.42597967220559144012987329549,0.0004431787299955088753610221267586369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,1620172800,2,1.0,-331.043259,1.000000e-01,0.000000,-80932,379427976363351507096,1385391852998664492209230571,0,DAI,WETH,3270.498157000950723102707230617198,0.0003057638170073151040382008689493288
2014,1620169200,1,NaN,NaN,NaN,NaN,-81234,379427976363351507096,1364573546772589608591144427,0,DAI,WETH,3371.050601932635214369473300707435,0.0002966434260662526005017999140948187
2015,1620165600,1,NaN,NaN,NaN,NaN,-81234,232295550235364197007,1364573546772589608591144427,0,DAI,WETH,3371.050601932635214369473300707435,0.0002966434260662526005017999140948187
2016,1620162000,2,1.0,-0.000336,1.000000e-07,0.000000,-81234,229713263233648638236,1364573546772589608591144427,0,DAI,WETH,3371.050601932635214369473300707435,0.0002966434260662526005017999140948187


In [20]:
# Eye check:
print(df_poolHourDatas.shape)
print(df_swaps_all.shape)
print(df_merged.shape)

print(df_merged['txCount'].sum())
print(df_merged['swaps_txCount'].sum())

(2018, 20)
(41937, 14)
(2018, 24)
49489
41937.0


In [22]:
df_merged.head()

,close,feeGrowthGlobal0X128,feeGrowthGlobal1X128,feesUSD,high,liquidity,low,open,periodStartUnix,sqrtPrice,...,txCount,volumeToken0,volumeToken1,volumeUSD,pool.token0.symbol,pool.token1.symbol,amount0,amount1,amountUSD,swaps_txCount
0,2301.692508394563049389060612506888,762637520884472688888896101924530883651,304834799378335995767948817935780294,0,2301.692508394563049389060612506888,2257757512531221050660252,2296.378085298876000615611994683579,2296.378085298876000615611994683579,1627466400,1651413884466706248548127541,...,6,0,0,0,DAI,WETH,NaN,NaN,NaN,NaN
1,2296.352739688466583961075436184692,762581680413962325638256491298510145295,304834799378335995767948817935780294,0,2296.352739688466583961075436184692,2258385571433368075249753,2283.693254559756255552465593941893,2283.693254559756255552465593941893,1627462800,1653332807396390538189673451,...,13,0,0,0,DAI,WETH,299616.218709,-130.443451,299557.018495,11.0
2,2283.693254559756255552465593941893,762446245049251001287289240454652627380,304834799378335995767948817935780294,0,2300.150695306308145058494739947916,2257382392731291728666040,2283.693254559756255552465593941893,2291.243722873524538168274057516396,1627459200,1657909038401687840269395027,...,23,0,0,0,DAI,WETH,-105151.549893,46.879335,671187.484515,16.0
3,2288.185835338192696952740469462558,762318356723728633586531017600678274911,304758009920411400262897603337522039,0,2288.185835338192696952740469462558,2256221622137200723238932,2256.900982636698665142702668584452,2256.900982636698665142702668584452,1627455600,1656280685119642923297705596,...,31,0,0,0,DAI,WETH,753317.520533,-330.534463,753167.996064,27.0
4,2256.42597967220559144012987329549,761977255467865845372200151046357926382,304757468258492808857271681961517064,0,2292.449181883422337580568690509998,2261588786047650735673230,2256.42597967220559144012987329549,2292.449181883422337580568690509998,1627452000,1667896272296713929624373234,...,17,0,0,0,DAI,WETH,-879730.423181,387.864660,881181.264433,17.0


In [23]:
df_swaps_all

,amount0,amount1,amountUSD,id,sqrtPriceX96,tick,timestamp,transaction.blockNumber,transaction.gasPrice,transaction.gasUsed,transaction.timestamp,periodStartUnix,periodEndUnix,swaps_txCount
0,39943.906498877751548863,-17.3487,39908.00999433632504975268725293466,0xfdba28865ede75ec8528a50e92af921ed384571c77f9...,1653332807396390538189673451,-77395,1627466383,12913772,39675000000,400000,1627466383,1627462800,1627466400,1
1,85978.509431782880080724,-37.386121486374052616,85951.0331674414440351218529941136,0x27774cf97e13ffad07d27eda47ea914994a3ca9971ef...,1653941430494411323362794497,-77388,1627466383,12913772,39675000001,1200000,1627466383,1627462800,1627466400,1
2,696.141880881646247754,-0.302946060907973564,695.8918824729876084303651016548295,0x1e8856fd39e69dc43d79386071a5935524030bf27148...,1655253001798429400665273829,-77372,1627465015,12913683,36000000000,230428,1627465015,1627462800,1627466400,1
3,998.539025777989356295,-0.434549624366996399,998.188221317852461237566066407408,0x5834bd7cc40f347075a399f08e289ec85d4b14d66757...,1655263629682370157283910736,-77372,1627465000,12913680,31000000000,299559,1627465000,1627462800,1627466400,1
4,40000,-17.414001698778369045,39993.44983704685965225836937487243,0x13d6b9121f5749af75919c5e933519208e876c834de7...,1655278874452216082905227799,-77372,1627464805,12913669,36000000000,458428,1627464805,1627462800,1627466400,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41932,10682.019392695845455577,-3.254228708039319328,10904.61988725889993324845984345543,0xb349853fe3e9c6069d8264202c9d68ad5b737919f0ca...,1383838353700367458448734141,-80954,1620237603,12375729,51000000000,1510740,1620237603,1620237600,1620241200,1
41933,-10.026667527943594868,0.00307643875786374,0,0x3d71709ade6b409b2223c6a3ef7356646f615d894802...,1386032315440134219844025713,-80922,1620177752,12371223,41500000000,330379,1620177752,1620176400,1620180000,1
41934,-331.043258762748247602,0.1,0,0x5cdf9ec984fb0fe801cc529c003bcdd5271857103c19...,1385391852998664492209230571,-80932,1620175296,12371051,36000000000,146605,1620175296,1620172800,1620176400,1
41935,-0.000336093753482074,0.0000001,0,0x1e268cf8d859b38f50d338aeaad6973b1335f5999f67...,1364573546772589608591144427,-81234,1620162681,12370126,58000000000,139064,1620162681,1620162000,1620165600,1


##### Query tokens with symbol
{
  tokens(first:10, where:{symbol: "WETH"}) {
    id
    symbol
    name
  }
}

##### Query pools with token0 id,  token1 ids and feeTiers
{
  pools(first:10, 
    where:{token0:"0x6b175474e89094c44da98b954eedeac495271d0f",
    token1: "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",
    feeTier:"3000" }) 
  {
    id
    token0{symbol}
    token1{symbol}
    feeTier
  }
}

##### Query poolDayDatas with pool id, order by date - Needs to be iterative (max 1000 query)
{
  poolDayDatas(first:1000,
  next: 1000,
    where:{pool:"0xa80964c5bbd1a0e95777094420555fead1a26c1e"},
  orderBy:date,
  orderDirection: desc) 
  {
    date
    tick
    liquidity
    volumeUSD
  }
}





##### Query examples on filtering

{
  pools
  (first: 10, 
    where: {liquidity_gt: "1000000", 
      feeTier: "10000"}
    orderBy: liquidity, 
    orderDirection: desc)
  {
    token0{symbol}
    token1{symbol}
    liquidity
  }


(token0) DAI id = 0x6b175474e89094c44da98b954eedeac495271d0f
(token1) WETH id = 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
(feeTier) "3000"

(DAI-WETH 500) Pool id = 0x60594a405d53811d3bc4766596efd80fd545a270
(DAI-WETH 3000) Pool id = 0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8
(DAI-WETH 1000) Pool id = 0xa80964c5bbd1a0e95777094420555fead1a26c1e




In [24]:
pool_id

'0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8'